In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation,TruncatedSVD
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  texts=[]
  classes=[]
  for i in range(len(corpus)):
    classes.append(corpus[i]['genre'])
    texts.append(re.sub("[^a-zA-Z0-9]+", ' ',corpus[i]['text']))
  
  
  
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(texts)
  X=X.toarray()

  print("The number of features ={}".format(len(list(vectorizer.vocabulary_.keys()))))
  return X,classes,list(vectorizer.vocabulary_.keys())

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):

  texts=[]
  classes=[]
  for i in range(len(corpus)):
    classes.append(corpus[i]['genre'])
    texts.append(re.sub("[^a-zA-Z0-9]+", ' ',corpus[i]['text']))
  
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(texts)
  X=X.toarray()

  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      if X[i][j] <=1:
        X[i][j] = 0
      else:
        X[i][j] = 1

  

  print("The number of features ={}".format(len(list(vectorizer.vocabulary_.keys()))))
  return X,classes,list(vectorizer.vocabulary_.keys())

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [0]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [0]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

The number of features =19797
----------L1 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['pounds', 'gums', 'minimus', 'rogero', 'mountebanks', 'elevated', 'immoment', 'rests', 'unspoke', 'wrist', 'couplement', 'guerra', 'fortuna', 'whereuntil', 'parfect', 'arinado', 'pursents', 'vara', 'career', 'jesting']
The most informative terms for neg are: ['beaks', 'cautelous', 'lading', 'brood', 'bemete', 'stubble', 'corse', 'vox', 'glowing', 'deserves', 'ber', 'simile', 'arinado', 'couplement', 'guerra', 'whereuntil', 'pompion', 'parfect', 'pursents', 'vara']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['rogero', 'gums', 'immoment', 'pounds', 'braving', 'rests', 'mountebanks', 'elevated', 'minimus', 'new', 'disarm', 'accompanied', 'goosequills', 'repeals', 'partially', 'despiseth', 'clad', 'hears', 'alexander', 'wrist']
The most informative terms for neg are: ['stubble', 'brood

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [0]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

The number of features =19797
----------L1 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['immoment', 'shrewish', 'either', 'handful', 'bere', 'targe', 'surnam', 'libbard', 'scutcheon', 'alls', 'couplement', 'guerra', 'fortuna', 'pompion', 'parfect', 'whereuntil', 'pursents', 'arinado', 'novum', 'alisander']
The most informative terms for neg are: ['thessalian', 'measurable', 'pleasers', 'tatters', 'grievous', 'wares', 'alisander', 'declares', 'scutcheon', 'bere', 'targe', 'surnam', 'scrap', 'libbard', 'pedlar', 'arinado', 'couplement', 'guerra', 'fortuna', 'pompion']
----------L2 Norm-----------
The model's average accuracy is 0.923077
The most informative terms for pos are: ['immoment', 'shrewish', 'either', 'juggling', 'steers', 'adulterate', 'complaining', 'thasos', 'superstitiously', 'ersways', 'receipts', 'handful', 'buzzers', 'pat', 'reckon', 'bidden', 'sailors', 'profession', 'goss', 'priam']
The most informative terms for neg 